In [37]:
from keras import backend as K
import tensorflow as tf
from sklearn.utils import check_random_state
from sklearn.random_projection import SparseRandomProjection
from sklearn.utils import check_random_state
import sklearn.utils  
import numpy as np
from sklearn.utils.random import sample_without_replacement
from numpy.testing import assert_equal
import scipy.sparse as sp

In [61]:
def project(x, ncomp, projecttype): #ncomp is the number of dimensions we want to shrink to
    
    features = K.int_shape(x)[1]
    
    if(projecttype == 'gaussian'):
        Y = gaussian_random_tensor(ncomp, features)
        
    else if(projecttype == 'sparse gaussian'):
        Y = sparse_random_tensor(ncomp, features)
        
    else if(projecttype == 'hadamard'):
        Y = hadamard_tensor(ncomp, features)
        
    X_new = K.dot(X, Y)
    return X_new

SyntaxError: invalid syntax (<ipython-input-61-dd3328639cdb>, line 8)

In [ ]:
def gaussian_random_tensor(n_components, n_features, random_state=None):
    rng = check_random_state(random_state)
    components = rng.normal(loc=0.0,
                            scale=1.0 / np.sqrt(n_components),
                            size=(n_components, n_features))
    return (tf.convert_to_tensor(components, dtype = 'float32'))  

In [28]:
def _check_density(density, n_features):
    """Factorize density check according to Li et al."""
    if density == 'auto':
        density = 1 / np.sqrt(n_features)

    elif density <= 0 or density > 1:
        raise ValueError("Expected density in range ]0, 1], got: %r"
                         % density)
    return density

In [55]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [60]:
def sparse_random_tensor(n_components, n_features, density='auto',
                         random_state=None):
    
    rng = check_random_state(random_state)
    density = _check_density(density, n_features)
    rng = check_random_state(random_state)

    if density == 1:
        # skip index generation if totally dense
        components = rng.binomial(1, 0.5, (n_components, n_features)) * 2 - 1
        return 1 / np.sqrt(n_components) * components

    else:
        # Generate location of non zero elements
        indices = []
        offset = 0
        indptr = [offset]
        for i in range(n_components):
            # find the indices of the non-zero components for row i
            n_nonzero_i = rng.binomial(n_features, density)
            indices_i = sample_without_replacement(n_features, n_nonzero_i,
                                                   random_state=rng)
            indices.append(indices_i)
            offset += n_nonzero_i
            indptr.append(offset)

        indices = np.concatenate(indices)

        # Among non zero components the probability of the sign is 50%/50%
        data = rng.binomial(1, 0.5, size=np.size(indices)) * 2 - 1

        # build the CSR structure by concatenating the rows
        components = sp.csr_matrix((data, indices, indptr),
                                   shape=(n_components, n_features))
        sparse = np.sqrt(1 / density) / np.sqrt(n_components) * components
        sparse = convert_sparse_matrix_to_sparse_tensor(sparse)
        return sparse
    

In [59]:
d = sparse_random_matrix(5, 5)
d

In [ ]:
from scipy.linalg import hadamard

def hadamard_tensor(n_components):
    
    had = hadamard(n_components, dtype=complex)
    
    return (tf.convert_to_tensor(had, dtype = 'float32'))  